In [1]:
import os
import numpy
from pathlib import Path

# ==========================================
# 🛠️ 3D SCANNER: THE FINAL REDIRECTION
# ==========================================

print("🧹 1. เคลียร์พื้นที่...")
if os.path.exists("3DSCAN"):
    !rm -rf 3DSCAN
!git clone https://github.com/PRIDA-TAKON/3DSCAN.git
os.chdir("3DSCAN")

# ---------------------------------------------------------

print("\n💊 2. สร้างป้ายบอกทาง (Redirection) ใน Numpy...")
try:
    # ไปที่ไฟล์เก่า (ห้องเก็บของ)
    stride_tricks_path = Path(numpy.__file__).parent / "lib" / "stride_tricks.py"
    
    with open(stride_tricks_path, "r") as f:
        original_content = f.read()
    
    # เขียนคำสั่ง: "ถ้าเรียก broadcast_to ให้ไปใช้ numpy.broadcast_to ตัวหลักแทน"
    # ใช้การ import ภายในฟังก์ชัน (Lazy Import) เพื่อป้องกัน Error งูกินหาง
    redirect_code = """

# --- KAGGLE FIX: REDIRECT TO MAIN NUMPY ---
def broadcast_to(array, shape, subok=False):
    import numpy as _np
    return _np.broadcast_to(array, shape, subok=subok)
# ------------------------------------------
"""
    
    if "def broadcast_to" not in original_content:
        with open(stride_tricks_path, "a") as f:
            f.write(redirect_code)
        print(f"✅ ติดตั้งป้ายบอกทางเรียบร้อยที่: {stride_tricks_path}")
    else:
        print("✅ มีป้ายบอกทางอยู่แล้ว")

except Exception as e:
    print(f"❌ Error: {e}")

# ---------------------------------------------------------

print("\n🔧 3. ปิด GPU Matching (กัน Crash)...")
!sed -i 's/"--SiftMatching.use_gpu", "1"/"--SiftMatching.use_gpu", "0"/' *.py
!sed -i 's/"--SiftExtraction.use_gpu", "1"/"--SiftExtraction.use_gpu", "0"/' *.py

# ---------------------------------------------------------

print("\n🚀 4. เริ่มรันโปรแกรม...")
import glob
py_files = glob.glob("*.py")
target_script = py_files[0] if py_files else "3d-scan.py"

!python "{target_script}" 2>&1 | tee 3d-scan.log.txt

🧹 1. เคลียร์พื้นที่...
Cloning into '3DSCAN'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 63 (delta 12), reused 5 (delta 3), pack-reused 30 (from 1)
Receiving objects: 100% (63/63), 436.17 KiB | 5.52 MiB/s, done.
Resolving deltas: 100% (22/22), done.

💊 2. สร้างป้ายบอกทาง (Redirection) ใน Numpy...
✅ ติดตั้งป้ายบอกทางเรียบร้อยที่: /usr/local/lib/python3.12/dist-packages/numpy/lib/stride_tricks.py

🔧 3. ปิด GPU Matching (กัน Crash)...

🚀 4. เริ่มรันโปรแกรม...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.4 requires fastcore<1.9,>=